<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Kinematic Wave Overland Flow Component 

## 1.0. Introduction

### 1.1. Theory

The 1-D Saint Venant equation for transient shallow water flow is in the core of most hydrodynamics models:

$$ \frac{\partial Q}{\partial t} + \frac{\partial}{\partial x}\left(\frac{Q^2}{A_{xs}}\right) + gA_{xs} \frac{\partial (h+z)}{\partial x} + \frac{gn^2 \lvert{Q}\rvert Q}{R^{4/3}A_{xs}} = 0 \space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space(1)$$
    


where      
&nbsp;&nbsp;&nbsp;&nbsp;$Q = discharge \left[L^3{T^{-1}}\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$t = time \left[T\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$x = location \space in \space space \left[L\right]$   
&nbsp;&nbsp;&nbsp;&nbsp;$g = acceleration \space due \space to \space gravity \left[L{T^{-2}}\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$h = water \space depth \left[L\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$z = bed \space elevation \left[L\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$n = Manning's \space roughness \space coeffiecient \left[T{L^{{-1}/{3}}}\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$R = hydraulic \space radius \left[L\right]$  
&nbsp;&nbsp;&nbsp;&nbsp;$A_{xs} = cross-sectional \space area \left[L^2\right]$ 

From left to right, the terms in Equation (1) represent **local acceleration, convective acceleration, gradients of fluid pressure and bed elevation, and friction**. Because this equation is difficult (i.e., almost impossible) to solve explicitly, approximations are commonly used. 

The simplest approximation, the **Kinematic Wave model**, only retains the **friction term**, making it the simplest approximation one can use. The Landlab `KinwaveImplicitOverlandFlow` component provides a 2-D locally implicit kinematic wave solution in which energy slope is assumed to be equal to the bed slope. 


This tutorial illustrates the use of `KinwaveImplicitOverlandFlow`. We will use:

* Spring Creek watershed, CO, USA 

and assume spatially uniform runoff generation and surface roughness (i.e., homogenous land cover and channel).

### Questions to consider before running this notebook

We will focus on the hydrograph characteristics, time to peak, peak discharge, and hydrograph shape. 

1. How do watershed shape and drainage area affect hydrograph characteristics? 
2. How does runoff intensity impact the time to peak and peak discharge across the channel network? 
3. How does runoff duration impact the time to peak and peak discharge across the channel network? 
4. How does surface roughness due to soil and land cover influence hydrograph peak discharge and shape?

**Import numpy and components:**

In [ ]:
import numpy as np
from time import time

from landlab.io.esri_ascii import read_esri_ascii

from landlab.components import KinwaveImplicitOverlandFlow, SinkFiller
from landlab.components.flow_accum import FlowAccumulator
#from landlab.components.flow_accum import find_drainage_area_and_discharge

from landlab.plot.imshow import imshow_grid
from landlab.plot.colors import water_colormap

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

# Set the environment to place plots in Jupyter cells.
%matplotlib inline

**Input parameters for KinwaveImplicitOverlandflow component**

In [ ]:
starting_precip_mmhr = 87    # [mm/hour] this is runoff rate uniformly distributed in space
storm_duration = 1.5         # [hour]
model_run_time = 4.0         # [hour]
    
n = 0.15                    # MAnning's roughness coefficient, (s/m^(1/3))

dt = 500                     # time step [sec] 

#Converting units to SI [m] and [sec]
storm_duration_sec=storm_duration*3600 # [sec]
model_run_time_sec=model_run_time*3600 # [sec]

**Import the DEM and find its outlet** 

In [ ]:
# Spring Creek Basin
(rmg, z) = read_esri_ascii('SpringCreek_DEM.asc', name='topographic__elevation')
rmg.set_watershed_boundary_condition(z)

**Derive drainage areas to select internal points for plotting hydrographs as needed.**

In [ ]:
# Spring Creek
fa= FlowAccumulator(rmg, flow_director='FlowDirectorMFD')  # Kinematicwave uses a multiple flow director algorithm
                                                           
fa.run_one_step()
(da, q) = fa.accumulate_flow()

**Select locations to plot hydrographs within ranges of drainage areas:** we would like to pick two points one in the middle and another towards the headwater region to compare hydrographs.

The midstream and upstream node drainge area thresholds can be identified by drainge area ranges in km$^2$. The range used to identify a midstream location should be larger than the range used to identify an upstream location. These internal nodes can be identified by trial using different drainge area values, and plotting the locations selected. After we select the drainage area ranges we will print the node IDs, drainge areas, and elevations of the locations we select and map the locations on the elevation map of the watershed. 

In [ ]:
# These values can be changed to set drainage area thresholds in km^2
midstream_da_upperbound = 20
midstream_da_lowerbound = 17.5
upstream_da_upperbound = 5
upstream_da_lowerbound = 3.2

# Identify sample nodes.
outlet_node_to_sample = np.argmax(rmg.at_node['drainage_area'])

midstream_node_to_sample = np.where(np.logical_and(rmg.at_node['drainage_area'] > 
                                                   midstream_da_lowerbound * 1000000, 
                                                   rmg.at_node['drainage_area']<midstream_da_upperbound*1000000))[0][0]

upstream_node_to_sample = np.where(np.logical_and(rmg.at_node['drainage_area'] > 
                                                  upstream_da_lowerbound*1000000, 
                                                  rmg.at_node['drainage_area']<upstream_da_upperbound*1000000))[0][0]

print('Outlet Node = ' + str(outlet_node_to_sample) + '; Drainage Area= ' + 
      str(da[outlet_node_to_sample] / 1000000) + ' km^2; Elev = '+ str(round(z[outlet_node_to_sample], 1)) + ' m')
print('Midstream Node = ' + str(midstream_node_to_sample) + '; Drainage Area= ' + 
      str(da[midstream_node_to_sample] / 1000000) + ' km^2; Elev = '+ str(round(z[midstream_node_to_sample], 1)) + ' m')
print('Upstream Node = ' + str(upstream_node_to_sample) + '; Drainage Area= ' + 
      str(da[upstream_node_to_sample] / 1000000) + ' km^2; Elev = '+ str(round(z[upstream_node_to_sample], 1)) + ' m')

**Plot the watershed and locations we chose above for plotting hydrographs**

In [ ]:
## Plot Spring Creek
# Plot the DEM.

# Set up the figure.
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
ax.xaxis.set_visible(False)
ax.set_facecolor("blue")

imshow_grid(rmg, z, plot_name='Spring Creek', var_name='topographic__elevation', var_units='m', grid_units=('m', 'm'), 
          cmap='terrain', limits=(1875, 2615), color_for_closed='white')


# Plot the sample nodes.
ax.plot(rmg.node_x[outlet_node_to_sample], rmg.node_y[outlet_node_to_sample], 'ro', label='outlet')
ax.plot(rmg.node_x[midstream_node_to_sample], rmg.node_y[midstream_node_to_sample], 'go', label='midstream')
ax.plot(rmg.node_x[upstream_node_to_sample], rmg.node_y[upstream_node_to_sample], 'bo', label='upstream')

_ = ax.legend(loc='lower right')

**Create empty lists for saving hydrographs at these location:**

In [ ]:
# Create lists for saving discharge and time data for plotting hydrographs.
discharge_at_outlet = [] 
discharge_midstream = []
discharge_upstream = []

hydrograph_time = []

# create an initial surface water depth field if you would like to map water depths
rmg.add_zeros("surface_water__depth", at="node", clobber=True)

**Now instantiate the component itself:** you can glance over the component before we run it by uncommenting the line below. 

In [ ]:
#KinwaveImplicitOverlandFlow?

In [ ]:
kw = KinwaveImplicitOverlandFlow(rmg, runoff_rate=0.0, roughness=n, depth_exp=5/3)

**Now we're going to run the loop that drives the component:** Elapsed time starts at 1 second. This prevents errors when running the component.

In [ ]:
elapsed_time=1;
#dt=1
while elapsed_time < model_run_time_sec:  
    
    if elapsed_time < storm_duration_sec:
        kw.runoff_rate=starting_precip_mmhr #This needs to be in mm/hr because the source code automatically converts to m/s
    else:
        kw.runoff_rate=1e-20                #This needs to be > 0 because of an assertion in the source code... 
        
    kw.run_one_step(dt) 
    
    # add elapsed time to the continuos time
    hydrograph_time.append(elapsed_time/3600)    
    
    # pick discharge values from nodes identified earlier and store them for plotting
    discharge_at_outlet.append(rmg.at_node['surface_water_inflow__discharge'][outlet_node_to_sample])
    discharge_midstream.append(rmg.at_node['surface_water_inflow__discharge'][midstream_node_to_sample])
    discharge_upstream.append(rmg.at_node['surface_water_inflow__discharge'][upstream_node_to_sample])
     
    ## output time every now and then so that you know the code
    ## is actually running
    if (elapsed_time % 100) < 2:
        print("elapsed time = ", elapsed_time)
        
    ## Updating elapsed_time  
    elapsed_time += dt

**Plot hydrographs:**

In [ ]:
plt.figure(1)
plt.plot(hydrograph_time, discharge_at_outlet, "r-", label="outlet")
plt.plot(hydrograph_time, discharge_midstream, "g-", label="midstream")
plt.plot(hydrograph_time, discharge_upstream, "b-", label="upstream")


plt.ylabel("Discharge (cms)")
plt.xlabel("Time (hour)")
plt.legend(loc="upper right")
title_text = "Hydrographs at three locations"
plt.title(title_text)
plt.show()

We can plot some **snapshots of water depth** on the domain. There is room to improve this plot.

In [ ]:
elapsed_time = 1.
run_time_slices = (1.2*3600, 1.25*3600)
for t in run_time_slices:
    while elapsed_time < t:
         # First, we calculate our time step.
        kw.run_one_step(dt) 

#         # Increased elapsed time
        elapsed_time += dt 
    figure(t)
    imshow_grid(rmg, 'surface_water__depth', cmap='Blues')

Try to explore the plot a little bit to see a better view

In [ ]:
imshow_grid(rmg,'surface_water__depth', plot_name = 'Surface water depth', 
            var_name = 'Depth of water', var_units = 'm', grid_units = ('m','m'), 
            cmap = 'jet', limits = (0, 5))

### Click here for more <a href="https://landlab.readthedocs.io/en/v2_dev/user_guide/tutorials.html">Landlab tutorials</a>